# Aplikacja w produkcji

## Jak dane dla tego scenariusza zostały pobrane

Odniesienie do kodu.

## Model w produkcji

Polegałbym na [fastapi](https://fastapi.tiangolo.com/) zamiast [flask](https://flask.palletsprojects.com/en/3.0.x/):

```python
from typing import Optional
from po_pr_predict import app
from pydantic import BaseModel
from fastai.tabular.all import load_learner
import joblib
import pandas as pd


learn = load_learner("./models/tabular_learner_1.pki")
model_ver = "0.1.0"


class Eban(BaseModel):
    quantity_requested: float
    valuation_price: float


class Predicton(BaseModel):
    banpr: str
    version: str


@app.post("/predict", response_model=Predicton)
def predict(eban: Eban):
    x = pd.Series(
        data={
            "Quantity Requested": eban.quantity_requested,
            "Valuation Price": eban.valuation_price,
        }
    )

    _, clas, _ = learn.predict(x)
    vocab = learn.dls.vocab
    r = int(vocab[clas])

    # format with leading 0
    return {"banpr": str(r).zfill(2), "version": model_ver}
```

## Model w chmurze

Docker + GCloud Run:

```bash
make docker_build
make publish


gcloud run deploy wsb-stocks-api \
           --platform managed \
          --image gcr.io/wsb-mlproject/wsb-stocks-api:0.1.0 \
          --port=8000 \
          --region europe-central2

```